# Import necessary libraries

In [1]:
import flywheel
import os
import re
import pandas as pd

## Set the criteria

In [ ]:

# How many subjects to process. If None, all subjects will be processed.
SUBJECT_LIMIT = 5

# Each of these are regular expressions that will be used to match the names
# of the files in each session.
t1_mask = r"_t1.nii.gz"
t1ce_mask = r"_t1ce.nii.gz"
t2_mask = r"_t2.nii.gz"
flair_mask = r"_flair.nii.gz"
seg_mask = r"_seg.nii.gz"

additional_img_mask = rf"{t1_mask}|{t1ce_mask}|{t2_mask}|{flair_mask}"
SEGMENT_NAMES = ["Necrosis", "Hypoxia", "Edema"]

## Initialize the Flywheel Client with a valid API key

In [3]:
api_key = os.getenv('FW_LATEST_KEY')
client = flywheel.Client(api_key)

# Get the project by id
project = client.get("654c48075129e1019eeecf83")

## Iterate Through the Subjects and Sessions to find qualifying cases

In [9]:
# Get and sort the subjects of the project
subjects = project.subjects()
sorted_subjects = sorted(subjects, key=lambda x: x.label)

# Rows of a CSV that will be saved locally and uploaded to the project
rows = []

if SUBJECT_LIMIT is None:
    SUBJECT_LIMIT = len(sorted_subjects)

# Iterate through subjects
for i in range(SUBJECT_LIMIT):
    subject = sorted_subjects[i]
    all_sessions = subject.sessions()

    # Iterate through sessions
    for session in all_sessions:
        # Each case consists of a primary image, a mask, additional images, and additional masks
        case_dict = {"additional_images": []}
        all_acquisitions = session.acquisitions()

        # Iterate through acquisitions
        # This is where we will match the files to the case criteria
        for acquisition in all_acquisitions:
            acquisition = acquisition.reload()
            files = acquisition.files
            for acq_file in files:
                img_dict = {
                        "parent": acquisition.id,
                        "name": acq_file.name
                    }
                if re.search(t1_mask, acq_file.name):
                    case_dict["image"] = img_dict
                elif re.search(additional_img_mask, acq_file.name):
                    case_dict["additional_images"].append(img_dict)
                elif re.search(seg_mask, acq_file.name):
                    img_dict["segment_names"] = SEGMENT_NAMES
                    case_dict["mask"] = img_dict
        primary_img_acq = client.get(case_dict["image"]["parent"])
        path = (
            f"fw://{project.label}/"
            f"{subject.label}/"
            f"{session.label}/"
            f"{primary_img_acq.label}"
        )
        rows.append({
            "path": path,
            "parent._id": primary_img_acq.id,
            "file.name": case_dict["image"]["name"]
        })
        primary_file = primary_img_acq.get_file(case_dict["image"]["name"])
        primary_file.update_info({"FlywheelCaseIteratorInfo": case_dict})



## Save the results to a CSV file

In [ ]:
case_df = pd.json_normalize(rows)
case_df.to_csv("case_dataframe.csv", index=False)
project.upload_file("case_dataframe.csv")